In [1]:
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.optimizers
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from keras.callbacks import EarlyStopping


import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
import os
import cv2
import glob
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_dir='/content/drive/MyDrive/DATA/final_data'

In [3]:
from tensorflow.keras.applications.mobilenet import MobileNet

base_mob = MobileNet(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

base_mob.trainable = False

17227776/17225924 [==============================] - 0s 0us/step


In [4]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

model = models.Sequential()
model.add(layers.InputLayer(input_shape=(224,224,3)))
model.add(base_mob)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

callback_1 = ModelCheckpoint('model.pt',
							  monitor='val_acc',
							  save_best_only = True,
							  mode='max',
							  verbose=2)
callback_2 = EarlyStopping(patience = 15)
callback_list = [callback_1, callback_2]

model.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [5]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        base_dir,
        subset='training',
        target_size=(224, 224),
        seed=0,
        batch_size=10,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        base_dir,
        subset='validation',
        target_size=(224, 224),
        seed=0,
        batch_size=10,
        class_mode='categorical')

Found 2499 images belonging to 5 classes.
Found 499 images belonging to 5 classes.


In [6]:
history = model.fit(
      train_generator,
      steps_per_epoch=20,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=10,
      verbose=2,
      callbacks=callback_list)

Epoch 1/50
20/20 - 134s - loss: 6.6973 - acc: 0.4200 - val_loss: 0.9167 - val_acc: 0.6700

Epoch 00001: val_acc improved from -inf to 0.67000, saving model to model.pt
INFO:tensorflow:Assets written to: model.pt/assets
Epoch 2/50
20/20 - 92s - loss: 0.9535 - acc: 0.6600 - val_loss: 1.0945 - val_acc: 0.5400

Epoch 00002: val_acc did not improve from 0.67000
Epoch 3/50
20/20 - 82s - loss: 0.7605 - acc: 0.7300 - val_loss: 0.4912 - val_acc: 0.8200

Epoch 00003: val_acc improved from 0.67000 to 0.82000, saving model to model.pt
INFO:tensorflow:Assets written to: model.pt/assets
Epoch 4/50
20/20 - 84s - loss: 0.5947 - acc: 0.7650 - val_loss: 0.2987 - val_acc: 0.9300

Epoch 00004: val_acc improved from 0.82000 to 0.93000, saving model to model.pt
INFO:tensorflow:Assets written to: model.pt/assets
Epoch 5/50
20/20 - 63s - loss: 0.5610 - acc: 0.7600 - val_loss: 0.3438 - val_acc: 0.8485

Epoch 00005: val_acc did not improve from 0.93000
Epoch 6/50
20/20 - 51s - loss: 0.6393 - acc: 0.7800 - val_l

In [7]:
train_datagen_aug = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2,
      rotation_range=50,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_generator_aug = train_datagen_aug.flow_from_directory(
        base_dir,
        subset='training',
        target_size=(224, 224),
        seed=0,
        batch_size=10,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        base_dir,
        subset='validation',
        target_size=(224, 224),
        seed=0,
        batch_size=10,
        class_mode='categorical')

Found 2000 images belonging to 5 classes.
Found 499 images belonging to 5 classes.


In [8]:
history_aug = model.fit(
      train_generator_aug,
      steps_per_epoch=20,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=10,
      verbose=2,
      callbacks=callback_list)

Epoch 1/50
20/20 - 6s - loss: 0.9049 - acc: 0.7450 - val_loss: 0.0766 - val_acc: 0.9800

Epoch 00001: val_acc did not improve from 1.00000
Epoch 2/50
20/20 - 5s - loss: 0.5792 - acc: 0.7850 - val_loss: 0.0452 - val_acc: 0.9900

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/50
20/20 - 5s - loss: 0.4897 - acc: 0.8000 - val_loss: 0.0968 - val_acc: 0.9700

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/50
20/20 - 5s - loss: 0.6614 - acc: 0.7550 - val_loss: 0.1353 - val_acc: 0.9600

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/50
20/20 - 6s - loss: 0.4528 - acc: 0.8400 - val_loss: 0.2724 - val_acc: 0.8889

Epoch 00005: val_acc did not improve from 1.00000
Epoch 6/50
20/20 - 5s - loss: 0.4435 - acc: 0.8150 - val_loss: 0.1797 - val_acc: 0.9100

Epoch 00006: val_acc did not improve from 1.00000
Epoch 7/50
20/20 - 6s - loss: 0.5653 - acc: 0.7950 - val_loss: 0.1635 - val_acc: 0.9100

Epoch 00007: val_acc did not improve from 1.00000
Epoch 8/50
20/20 - 6s - los